### 🎯 Preprocessing & Feature Extraction

#### ✅ MOV

In [ ]:
# =========================================================
# 🎯 SeizeIT2 MOV — Motion Features (ACC + GYR Magnitudes Only)
#    ✅ Works only on runs listed in seizure_runs_details.csv
#    ✅ Labels: 0 = No seizure, 1 = 15 min before seizure (Pre-ictal), 2 = Seizure (Ictal)
#    ✅ Missing channels → their features are set to NaN
# =========================================================

import os, glob, json, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import mne

warnings.filterwarnings("ignore")
mne.set_log_level("WARNING")

# =========================================================
# 📂 PATHS & CONSTANTS
# =========================================================
BASE_ROOT = r"C:\SeniorProject_Data"
OUT_ROOT  = r"C:\SeniorProject\Good_data"
os.makedirs(OUT_ROOT, exist_ok=True)

SESSION      = "ses-01"
EPOCH_SEC    = 2.0
STEP_SEC     = EPOCH_SEC
PREICTAL_SEC = 15 * 60
HP_HZ, LP_HZ = 0.3, 10.0

CSV_ALL = os.path.join(OUT_ROOT, "MOV_Good_Runs.csv")

# =========================================================
# 📄 Load good runs file only
# =========================================================
GOOD_RUNS_CSV = r"C:\SeniorProject\Good_Data\seizure_runs_details.csv"

print("\n============================")
print("📄 Loading good runs list ...")
good_df = pd.read_csv(GOOD_RUNS_CSV)
print("Loaded good runs:", good_df.shape)
print("Columns:", list(good_df.columns))

required_cols = {"subject", "run"}
if not required_cols.issubset(good_df.columns):
    raise ValueError(f"❌ Good runs file must contain the columns: {required_cols}")

good_df["run"] = good_df["run"].astype(int)
good_pairs = set(zip(good_df["subject"], good_df["run"]))
print(f"✅ Number of (subject, run) pairs in GOOD list: {len(good_pairs)}")

subjects = sorted(good_df["subject"].unique())
print("Subjects to process:", subjects)

# =========================================================
# 🔁 Load existing CSV to detect processed (sub, run_base)
# =========================================================
done_pairs = set()
if os.path.exists(CSV_ALL):
    try:
        df_done = pd.read_csv(CSV_ALL, usecols=["subject", "run_base"])
        done_pairs = set(zip(df_done["subject"], df_done["run_base"]))
        print(f"🔁 Existing rows: {len(df_done)}  |  Processed runs: {len(done_pairs)}")
    except Exception as e:
        print("⚠ Error reading existing CSV, will start fresh:", e)
        done_pairs = set()
else:
    print("ℹ No existing CSV file. Fresh start.")

# =========================================================
# 🧩 Build state mask (labels 0 / 1 / 2)
# =========================================================
def build_state_mask_from_events(sig_len, fs, events_path, preictal_sec=900):
    """
    Returns a mask array of length = number of samples:
    0 = no seizure (Normal)
    1 = pre-ictal (preictal_sec seconds before seizure onset)
    2 = ictal (seizure period)
    """
    mask = np.zeros(sig_len, dtype=np.int8)

    if (events_path is None) or (not os.path.exists(events_path)):
        return mask

    df = pd.read_csv(events_path, sep="\t")

    # If duration is missing, use ON/OFF seizure events
    if "duration" not in df.columns or df["duration"].isna().all():
        onsets, offsets = [], []
        for _, row in df.iterrows():
            ev = str(row.get("eventType", "")).lower()
            if "onset" in ev and "sz" in ev:
                onsets.append(float(row["onset"]))
            elif "offset" in ev and "sz" in ev:
                offsets.append(float(row["onset"]))
        for on_s, off_s in zip(onsets, offsets):
            on = int(on_s * fs)
            off = int(off_s * fs)
            mask[on:off] = 2
            pre = max(0, on - int(preictal_sec * fs))
            mask[pre:on] = 1
        return mask

    # If we have onset + duration columns
    for _, row in df.iterrows():
        ev = str(row.get("eventType", "")).lower()
        if "sz" not in ev:
            continue
        on = int(float(row["onset"]) * fs)
        off = int((float(row["onset"]) + float(row["duration"])) * fs)
        mask[on:off] = 2
        pre = max(0, on - int(preictal_sec * fs))
        mask[pre:on] = 1

    return mask

# =========================================================
# ⚡ Frequency features (with NaN support)
# =========================================================
def compute_freq_features(x, fs):
    x = np.asarray(x, float)
    N = len(x)

    if N < 2 or np.all(np.isnan(x)):
        return {
            "spec_energy":     np.nan,
            "spec_entropy":    np.nan,
            "band_3_10_power": np.nan,
            "peak_freq":       np.nan,
        }

    x = x[~np.isnan(x)]
    if len(x) < 2:
        return {
            "spec_energy":     np.nan,
            "spec_entropy":    np.nan,
            "band_3_10_power": np.nan,
            "peak_freq":       np.nan,
        }

    N = len(x)
    x = x - np.mean(x)
    psd = (np.abs(np.fft.rfft(x))**2) / N
    freqs = np.fft.rfftfreq(N, d=1/fs)

    total = np.sum(psd) + 1e-12
    entropy = -np.sum((psd/total) * np.log2(psd/total + 1e-12))
    band = np.sum(psd[(freqs >= 3) & (freqs <= 10)])
    peak = freqs[np.argmax(psd)]

    return {
        "spec_energy":     float(total),
        "spec_entropy":    float(entropy),
        "band_3_10_power": float(band),
        "peak_freq":       float(peak),
    }

# =========================================================
# 🧮 Extract features per window (with NaN for missing channels)
# =========================================================
def compute_features_for_window_mag(win_mag_dict, fs, idx, label, t0, t1):
    dt = 1.0 / fs
    row = {
        "win_idx": idx,
        "t_start": t0,
        "t_end": t1,
        "timestamp_center": (t0 + t1) / 2,
        "label": int(label),
    }

    for name, x in win_mag_dict.items():
        x = np.asarray(x, float)

        if np.all(np.isnan(x)) or len(x) < 2:
            std = rms = energy = var = rng = jerk = np.nan
            freq = {
                "spec_energy":     np.nan,
                "spec_entropy":    np.nan,
                "band_3_10_power": np.nan,
                "peak_freq":       np.nan,
            }
        else:
            x_valid = x[~np.isnan(x)]
            if len(x_valid) < 2:
                std = rms = energy = var = rng = jerk = np.nan
                freq = {
                    "spec_energy":     np.nan,
                    "spec_entropy":    np.nan,
                    "band_3_10_power": np.nan,
                    "peak_freq":       np.nan,
                }
            else:
                std    = np.std(x_valid)
                rms    = np.sqrt(np.mean(x_valid**2))
                energy = np.sum(x_valid**2)
                var    = np.var(x_valid)
                rng    = np.max(x_valid) - np.min(x_valid)
                jerk   = np.sqrt(np.mean((np.diff(x_valid) / dt) ** 2))

                freq = compute_freq_features(x_valid, fs)

        p = name
        row[f"{p}_std"]             = std
        row[f"{p}_rms"]             = rms
        row[f"{p}_energy"]          = energy
        row[f"{p}_var"]             = var
        row[f"{p}_range"]           = rng
        row[f"{p}_jerk_rms"]        = jerk
        row[f"{p}_band_3_10_power"] = freq["band_3_10_power"]
        row[f"{p}_peak_freq"]       = freq["peak_freq"]
        row[f"{p}_spec_entropy"]    = freq["spec_entropy"]

    return row

# =========================================================
# 🔧 Helpers for channels
# =========================================================
def safe_pick_channels(ch_names, include_keywords):
    picked = [i for i, n in enumerate(ch_names)
              if all(k in n for k in include_keywords)]
    return picked if len(picked) > 0 else None

def extract_or_nan(idx_list, data_arr, n_samples):
    if idx_list is None:
        return np.full((1, n_samples), np.nan, dtype=float)
    else:
        return data_arr[idx_list, :]

# =========================================================
# 🚀 MAIN — process only runs listed in good_pairs
# =========================================================
for SUBJ_ID in subjects:

    mov_path = os.path.join(BASE_ROOT, SUBJ_ID, SESSION, "mov")
    if not os.path.exists(mov_path):
        print(f"\n⚠ No MOV folder for {SUBJ_ID}, skipping…")
        continue

    mov_files = sorted(glob.glob(os.path.join(mov_path, "*_mov.edf")))
    if not mov_files:
        print(f"\n⚠ No MOV files for {SUBJ_ID}, skipping…")
        continue

    total_runs = len(mov_files)
    print("\n============================")
    print(f"Subject: {SUBJ_ID} — Runs in folder: {total_runs}")

    for run_i, mov_edf in enumerate(mov_files, start=1):

        base = os.path.basename(mov_edf).replace("_mov.edf", "")

        try:
            run_str = base.split("run-")[-1]
            run_num = int(run_str)
        except Exception as e:
            print(f"⚠ Can't parse run number from {base}, skipping…")
            continue

        if (SUBJ_ID, run_num) not in good_pairs:
            continue

        if (SUBJ_ID, base) in done_pairs:
            print(f"⏭ Run {run_i}/{total_runs} — SKIPPED (already saved): {SUBJ_ID} | {base}")
            continue

        print(f"\n➡ Run {run_i}/{total_runs} — Loading (GOOD): {base} (run={run_num})")

        eeg_dir = os.path.join(os.path.dirname(os.path.dirname(mov_edf)), "eeg")
        events_path = os.path.join(eeg_dir, f"{base}_events.tsv")
        if not os.path.exists(events_path):
            alt = os.path.join(eeg_dir, f"{base}_eeg_events.tsv")
            events_path = alt if os.path.exists(alt) else None

        raw = mne.io.read_raw_edf(mov_edf, preload=True)
        fs  = raw.info["sfreq"]
        data = raw.get_data()
        ch_names = raw.ch_names
        n_ch, n_samp = data.shape

        print(f"Fs={fs}, Channels={n_ch}, Samples={n_samp}")

        raw.filter(HP_HZ, LP_HZ, picks="all", method="iir")

        data = raw.get_data()
        data = (data - data.mean(axis=1, keepdims=True)) / (data.std(axis=1, keepdims=True) + 1e-12)

        n_samples = raw.n_times

        EEG_ACC_idx = safe_pick_channels(ch_names, ["EEG SD", "ACC"])
        EEG_GYR_idx = safe_pick_channels(ch_names, ["EEG SD", "GYR"])
        ECG_ACC_idx = safe_pick_channels(ch_names, ["ECGEMG SD", "ACC"])
        ECG_GYR_idx = safe_pick_channels(ch_names, ["ECGEMG SD", "GYR"])

        EEG_ACC_data = extract_or_nan(EEG_ACC_idx, data, n_samples)
        EEG_GYR_data = extract_or_nan(EEG_GYR_idx, data, n_samples)
        ECG_ACC_data = extract_or_nan(ECG_ACC_idx, data, n_samples)
        ECG_GYR_data = extract_or_nan(ECG_GYR_idx, data, n_samples)

        mask = build_state_mask_from_events(n_samp, fs, events_path, PREICTAL_SEC)

        def full_mag(arr):
            return np.linalg.norm(arr, axis=0)

        mags = []
        names = []

        mags.append(full_mag(EEG_ACC_data));    names.append("EEG_ACC_MAG")
        mags.append(full_mag(EEG_GYR_data));    names.append("EEG_GYR_MAG")
        mags.append(full_mag(ECG_ACC_data));    names.append("ECGEMG_ACC_MAG")
        mags.append(full_mag(ECG_GYR_data));    names.append("ECGEMG_GYR_MAG")

        mags = np.vstack(mags)

        # ---- Windowing ----
        win = int(EPOCH_SEC * fs)
        starts = np.arange(0, n_samp - win + 1, win)

        rows = []
        for w_i, s0 in enumerate(starts):
            s1 = s0 + win
            t0, t1 = s0 / fs, s1 / fs

            # 🔴 This is the only modification: label priority rule (like EMG pipeline)
            win_mask = mask[s0:s1]
            if (win_mask == 2).any():
                lbl = 2
            elif (win_mask == 1).any():
                lbl = 1
            else:
                lbl = 0
            # Previously: mid = int((s0 + s1) / 2); lbl = int(mask[mid])

            win_mag = {name: sig[s0:s1] for sig, name in zip(mags, names)}
            r = compute_features_for_window_mag(win_mag, fs, w_i, lbl, t0, t1)
            r["subject"]  = SUBJ_ID
            r["run_base"] = base
            rows.append(r)

        df = pd.DataFrame(rows)

        labels = df["label"].values
        unique, counts = np.unique(labels, return_counts=True)

        print("\n[Window Counts]")
        for u, c in zip(unique, counts):
            icon = {0: "🟦 Normal", 1: "🟨 Pre-ictal", 2: "🟥 Ictal"}.get(u, str(u))
            print(f"   {icon}: {c} windows")

        print(f"➡ Finished Run {run_i}/{total_runs} — Total windows: {len(rows)}")

        if os.path.exists(CSV_ALL):
            df.to_csv(CSV_ALL, mode='a', header=False, index=False)
            # print(f"➕ Appended {len(df)} rows → {CSV_ALL}")
        else:
            df.to_csv(CSV_ALL, mode='w', header=True, index=False)
            # print(f"🆕 Created → {CSV_ALL}")

        done_pairs.add((SUBJ_ID, base))

print("\n\n🎉 DONE — processed ONLY runs listed in seizure_runs_details.csv")
print(f"Final CSV → {CSV_ALL}")



📄 Loading good runs list ...
Loaded good runs: (883, 6)
Columns: ['subject', 'session', 'run', 'onset', 'duration', 'eventType']
✅ Number of (subject, run) pairs in GOOD list: 395
Subjects to process: ['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-047', 'sub-048', 'sub-049', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-069', 'sub-070', 'sub-071', 'sub-072', 'sub-

In [8]:
import pandas as pd
MOV = pd.read_csv(r"C:\SeniorProject\Good_Data\MOV_Good_Runs.csv")
MOV.shape


(5411097, 43)

In [10]:
MOV.columns

Index(['win_idx', 't_start', 't_end', 'timestamp_center', 'label',
       'EEG_ACC_MAG_std', 'EEG_ACC_MAG_rms', 'EEG_ACC_MAG_energy',
       'EEG_ACC_MAG_var', 'EEG_ACC_MAG_range', 'EEG_ACC_MAG_jerk_rms',
       'EEG_ACC_MAG_band_3_10_power', 'EEG_ACC_MAG_peak_freq',
       'EEG_ACC_MAG_spec_entropy', 'EEG_GYR_MAG_std', 'EEG_GYR_MAG_rms',
       'EEG_GYR_MAG_energy', 'EEG_GYR_MAG_var', 'EEG_GYR_MAG_range',
       'EEG_GYR_MAG_jerk_rms', 'EEG_GYR_MAG_band_3_10_power',
       'EEG_GYR_MAG_peak_freq', 'EEG_GYR_MAG_spec_entropy',
       'ECGEMG_ACC_MAG_std', 'ECGEMG_ACC_MAG_rms', 'ECGEMG_ACC_MAG_energy',
       'ECGEMG_ACC_MAG_var', 'ECGEMG_ACC_MAG_range', 'ECGEMG_ACC_MAG_jerk_rms',
       'ECGEMG_ACC_MAG_band_3_10_power', 'ECGEMG_ACC_MAG_peak_freq',
       'ECGEMG_ACC_MAG_spec_entropy', 'ECGEMG_GYR_MAG_std',
       'ECGEMG_GYR_MAG_rms', 'ECGEMG_GYR_MAG_energy', 'ECGEMG_GYR_MAG_var',
       'ECGEMG_GYR_MAG_range', 'ECGEMG_GYR_MAG_jerk_rms',
       'ECGEMG_GYR_MAG_band_3_10_power', 'ECGEMG_

In [11]:
# طباعة الشكل
print("MOV shape:", MOV.shape)

# عدد السبجكت
print("\n🟦 Unique subjects:", MOV['subject'].nunique())
print(MOV['subject'].unique())

# عدد الرنز (run_base)
print("\n🟩 Unique runs:", MOV['run_base'].nunique())
print(MOV['run_base'].unique())

# لكل سبجكت كم رن عنده
print("\n📌 Runs per subject:")
runs_per_sub = MOV.groupby('subject')['run_base'].nunique()
print(runs_per_sub)

# جدول كامل subject - run_base - count
runs_df = MOV.groupby(['subject', 'run_base']).size().reset_index(name='count')
print("\n📋 Detailed table (subject, run_base, count):")
print(runs_df)


MOV shape: (5411097, 43)

🟦 Unique subjects: 117
['sub-001' 'sub-002' 'sub-003' 'sub-004' 'sub-005' 'sub-006' 'sub-007'
 'sub-008' 'sub-009' 'sub-010' 'sub-011' 'sub-012' 'sub-013' 'sub-014'
 'sub-015' 'sub-016' 'sub-017' 'sub-018' 'sub-019' 'sub-020' 'sub-021'
 'sub-022' 'sub-023' 'sub-024' 'sub-025' 'sub-026' 'sub-027' 'sub-028'
 'sub-029' 'sub-030' 'sub-031' 'sub-032' 'sub-033' 'sub-034' 'sub-035'
 'sub-036' 'sub-037' 'sub-038' 'sub-039' 'sub-040' 'sub-041' 'sub-042'
 'sub-043' 'sub-044' 'sub-045' 'sub-046' 'sub-047' 'sub-049' 'sub-050'
 'sub-051' 'sub-052' 'sub-053' 'sub-054' 'sub-055' 'sub-056' 'sub-057'
 'sub-058' 'sub-059' 'sub-060' 'sub-061' 'sub-062' 'sub-063' 'sub-064'
 'sub-065' 'sub-066' 'sub-067' 'sub-068' 'sub-069' 'sub-070' 'sub-071'
 'sub-072' 'sub-073' 'sub-074' 'sub-075' 'sub-076' 'sub-077' 'sub-078'
 'sub-079' 'sub-080' 'sub-083' 'sub-084' 'sub-085' 'sub-086' 'sub-088'
 'sub-089' 'sub-092' 'sub-093' 'sub-094' 'sub-095' 'sub-096' 'sub-097'
 'sub-098' 'sub-099' 'sub-10

In [ ]:
import pandas as pd

# Load the CSV file
Test = pd.read_csv(r"C:\SeniorProject\Good_Data\seizure_runs_details.csv")

# Print the shape of the dataframe
print("MOV shape:", Test.shape)

# Full table: subject - run - count
runs_df = Test.groupby(['subject', 'run']).size().reset_index(name='count')
print(runs_df)


MOV shape: (883, 6)
     subject  run  count
0    sub-001    3      1
1    sub-001    5      1
2    sub-001    7      1
3    sub-001    8      1
4    sub-002    1      2
..       ...  ...    ...
390  sub-123   50      1
391  sub-123   81      1
392  sub-124   47      1
393  sub-125   21      1
394  sub-125   37      1

[395 rows x 3 columns]


#### ✅ EEG

##### RUN: 201-395

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import pywt

mne.set_log_level("WARNING")

# =========================================
# 0) Paths + constants
# =========================================

# Be careful with the dataset path (it contains SeizeIT2\SeizeIT2)
BASE_DIR = Path(r"C:\SeniorProject_Data")

# File containing runs + seizure info (CSV with onset/duration per run)
RUNS_CSV = Path(r"C:\SeniorProject\Good_Data\seizure_runs_details.csv")

# Output CSV file for Batch 1
OUT_FEAT_CSV = Path(r"C:\SeniorProject\Good_Data\MOV_Good_Runs_Batch201-395.csv")

# Constants
WINDOW_SEC   = 2.0          # Window length in seconds
PREICTAL_SEC = 15 * 60      # 15 minutes before seizure (label=1)

print("BASE_DIR:", BASE_DIR)
print("RUNS_CSV exists:", RUNS_CSV.exists())

# Load runs + seizures table from CSV
runs_table = pd.read_csv(RUNS_CSV)
runs_table["subject"] = runs_table["subject"].astype(str).str.strip()   # ex: sub-001
runs_table["session"] = runs_table["session"].astype(str).str.strip()   # ex: ses-01
runs_table["run"]     = pd.to_numeric(runs_table["run"], errors="raise").astype(int)

print("Runs table shape:", runs_table.shape)
display(runs_table.head())


# =========================================
# 1) Build the state mask from the runs CSV (instead of events.tsv)
# =========================================
def build_state_mask_from_runs(raw, subject, session, run, runs_df, preictal_sec=PREICTAL_SEC):
    sf = raw.info["sfreq"]
    n_samples = raw.n_times
    mask = np.zeros(n_samples, dtype=np.int8)

    sub_df = runs_df[
        (runs_df["subject"] == subject) &
        (runs_df["session"] == session) &
        (runs_df["run"] == int(run))
    ].copy()

    if sub_df.empty:
        return mask

    sub_df["onset"]    = pd.to_numeric(sub_df["onset"], errors="coerce")
    sub_df["duration"] = pd.to_numeric(sub_df["duration"], errors="coerce")
    sub_df = sub_df.dropna(subset=["onset", "duration"])

    if sub_df.empty:
        return mask

    for _, row in sub_df.iterrows():
        onset_sec = float(row["onset"])
        dur_sec   = float(row["duration"])

        ictal_start = int(onset_sec * sf)
        ictal_end   = int((onset_sec + dur_sec) * sf)

        ictal_start = max(0, min(ictal_start, n_samples))
        ictal_end   = max(0, min(ictal_end,   n_samples))

        pre_start = max(0, ictal_start - int(preictal_sec * sf))

        mask[pre_start:ictal_start] = 1
        mask[ictal_start:ictal_end] = 2

    return mask



# =========================================
# 2) ECG/EMG artefact helpers
# =========================================
def prepare_ref_for_regression(eeg_raw, ref_raw, ref_name="REF"):
    if ref_raw is None:
        print(f"[{ref_name}] No reference file.")
        return None

    sf_eeg = eeg_raw.info["sfreq"]
    sf_ref = ref_raw.info["sfreq"]

    ref = ref_raw.copy()
    if sf_ref != sf_eeg:
        print(f"[{ref_name}] Resampling reference from {sf_ref} Hz to {sf_eeg} Hz (reference only).")
        ref.resample(sfreq=sf_eeg, npad="auto")

    eeg_len = eeg_raw.n_times
    ref_data = ref.get_data()
    ref_len = ref_data.shape[1]

    if ref_len > eeg_len:
        ref_data = ref_data[:, :eeg_len]
    elif ref_len < eeg_len:
        pad = np.zeros((ref_data.shape[0], eeg_len - ref_len), dtype=ref_data.dtype)
        ref_data = np.concatenate([ref_data, pad], axis=1)

    if np.var(ref_data) < 1e-10:
        print(f"[{ref_name}] Reference signal is almost flat → skip.")
        return None

    return ref_data


def regression_artifact_removal(eeg_raw, ref_raw, ref_name="REF"):
    ref_data = prepare_ref_for_regression(eeg_raw, ref_raw, ref_name=ref_name)
    if ref_data is None:
        print(f"[{ref_name}] Regression skipped.")
        return eeg_raw.copy()

    Y = eeg_raw.get_data()
    X_ref = ref_data

    X = np.vstack([X_ref, np.ones((1, X_ref.shape[1]))]).T
    Y_T = Y.T

    beta, _, _, _ = np.linalg.lstsq(X, Y_T, rcond=None)
    artefact = (X @ beta).T
    cleaned  = Y - artefact

    out = eeg_raw.copy()
    out._data = cleaned
    print(f"[{ref_name}] Regression artefact removal done.")
    return out


def has_ecg_artefact(eeg_raw, ecg_raw, corr_thresh=0.3):
    ref_data = prepare_ref_for_regression(eeg_raw, ecg_raw, ref_name="ECG")
    if ref_data is None:
        return False

    eeg_data = eeg_raw.get_data()
    ref_ch   = ref_data[0]

    corrs = []
    for ch_idx in range(eeg_data.shape[0]):
        sig = eeg_data[ch_idx]
        c = np.corrcoef(sig, ref_ch)[0, 1]
        if not np.isnan(c):
            corrs.append(abs(c))

    if not corrs:
        return False

    max_corr = max(corrs)
    print(f"[ECG] max |corr(EEG, ECG)| = {max_corr:.3f}")
    return max_corr >= corr_thresh


def has_emg_artefact(eeg_raw, high_freq_band=(40.0, 100.0), ratio_thresh=0.3):
    data = eeg_raw.get_data()
    sf   = eeg_raw.info["sfreq"]

    hf = mne.filter.filter_data(
        data, sf,
        l_freq=high_freq_band[0],
        h_freq=high_freq_band[1],
        verbose=False
    )

    hf_power    = np.mean(hf**2, axis=1)
    total_power = np.mean(data**2, axis=1) + 1e-12
    ratios = hf_power / total_power
    median_ratio = float(np.median(ratios))

    print(f"[EMG] median HF(40–100Hz)/total power = {median_ratio:.3f}")
    return median_ratio >= ratio_thresh


def wavelet_denoise_emg_raw(eeg_raw, wavelet="db4", level=2, thr_scale=0.5):
    data = eeg_raw.get_data()
    denoised = np.zeros_like(data)

    for ch in range(data.shape[0]):
        sig = data[ch, :]
        coeffs = pywt.wavedec(sig, wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        thr = thr_scale * sigma * np.sqrt(2 * np.log(len(sig)))

        new_coeffs = [coeffs[0]]
        for d in coeffs[1:]:
            new_d = pywt.threshold(d, thr, mode="soft")
            new_coeffs.append(new_d)

        recon = pywt.waverec(new_coeffs, wavelet)
        if len(recon) > len(sig):
            recon = recon[:len(sig)]
        elif len(recon) < len(sig):
            pad = np.zeros(len(sig) - len(recon))
            recon = np.concatenate([recon, pad])
        denoised[ch, :] = recon

    out = eeg_raw.copy()
    out._data = denoised
    print("[EMG] Wavelet denoising done.")
    return out


# =========================================
# 3) window_and_label → PER-WINDOW with priority (2 → 1 → 0)
# =========================================
def window_and_label(eeg_raw, state_mask, win_sec=WINDOW_SEC):
    """
    - Segment the signal into 2-second non-overlapping windows
    - Assign each window a label based on the state_mask
    """
    sf = eeg_raw.info["sfreq"]
    data = eeg_raw.get_data()
    n_ch, n_samples = data.shape

    win_samples = int(win_sec * sf)
    if n_samples < win_samples:
        print("⚠ Run too short.")
        return None, None

    mask = np.asarray(state_mask, dtype=np.int8)
    if mask.size < n_samples:
        pad = np.zeros(n_samples - mask.size, dtype=np.int8)
        mask = np.concatenate([mask, pad])
    elif mask.size > n_samples:
        mask = mask[:n_samples]

    starts = np.arange(0, n_samples - win_samples + 1, win_samples)

    X_list, y_list = [], []
    for s in starts:
        e = s + win_samples
        seg = data[:, s:e]
        seg_mask = mask[s:e]

        if seg_mask.size == 0:
            lab = 0
        else:
            if (seg_mask == 2).any():
                lab = 2
            elif (seg_mask == 1).any():
                lab = 1
            else:
                lab = 0

        X_list.append(seg)
        y_list.append(lab)

    if not X_list:
        return None, None

    X = np.stack(X_list, axis=0)
    y = np.array(y_list, dtype=np.int8)

    print(f"[window_and_label] X shape = {X.shape}")
    print(f"[window_and_label] label counts = {dict(zip(*np.unique(y, return_counts=True)))}")
    return X, y


# =========================================
# 4) Feature helpers PER-CHANNEL
# =========================================
def compute_bandpowers_and_relative(x, sf):
    """
    Computes:
    - Absolute bandpowers: Delta, Theta, Alpha, Beta, Gamma
    - Relative bandpowers
    """
    x = np.asarray(x, dtype=float)
    n = x.size

    freqs = np.fft.rfftfreq(n, d=1.0 / sf)
    fft_vals = np.fft.rfft(x)
    psd = (np.abs(fft_vals) ** 2) / n

    def band_power(fmin, fmax):
        mask = (freqs >= fmin) & (freqs < fmax)
        if not np.any(mask):
            return 0.0
        return psd[mask].sum()

    bp_delta = band_power(0.5, 4.0)
    bp_theta = band_power(4.0, 8.0)
    bp_alpha = band_power(8.0, 12.0)
    bp_beta  = band_power(12.0, 30.0)
    bp_gamma = band_power(30.0, 45.0)

    total_bp = bp_delta + bp_theta + bp_alpha + bp_beta + bp_gamma + 1e-12

    return {
        "Delta Bandpower": bp_delta,
        "Theta Bandpower": bp_theta,
        "Alpha Bandpower": bp_alpha,
        "Beta Bandpower":  bp_beta,
        "Gamma Bandpower": bp_gamma,

        "Relative Delta Bandpower": bp_delta / total_bp,
        "Relative Theta Bandpower": bp_theta / total_bp,
        "Relative Alpha Bandpower": bp_alpha / total_bp,
        "Relative Beta Bandpower":  bp_beta  / total_bp,
        "Relative Gamma Bandpower": bp_gamma / total_bp,
    }


def compute_time_features(x):
    """
    Computes:
    - IQR, MAD, Mean, Median, Variance, Entropy
    - Standard Deviation, Skewness, Kurtosis
    - Line Length
    - Hjorth: activity, mobility, complexity
    """
    x = np.asarray(x, dtype=float)

    mean = np.mean(x)
    median = np.median(x)
    var = np.var(x)
    std = np.sqrt(var + 1e-12)

    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1

    mad = np.median(np.abs(x - median))

    if np.allclose(x, x[0]):
        entropy = 0.0
    else:
        hist, _ = np.histogram(x, bins=50, density=True)
        p = hist + 1e-12
        p = p / p.sum()
        entropy = -np.sum(p * np.log2(p))

    centered = x - mean
    m3 = np.mean(centered**3)
    m4 = np.mean(centered**4)
    skew = m3 / (std**3 + 1e-12)
    kurt = m4 / (std**4 + 1e-12)

    line_len = np.sum(np.abs(np.diff(x)))

    activity = var
    dx = np.diff(x)
    var_dx = np.var(dx)
    mobility = np.sqrt(var_dx / (var + 1e-12))

    ddx = np.diff(dx)
    var_ddx = np.var(ddx) if ddx.size > 0 else 0.0
    mobility_dx = np.sqrt(var_ddx / (var_dx + 1e-12)) if var_dx > 0 else 0.0
    complexity = mobility_dx / (mobility + 1e-12)

    return {
        "Interquartile Range": iqr,
        "Median Absolute Deviation": mad,
        "Mean": mean,
        "Median": median,
        "Variance": var,
        "Entropy": entropy,
        "Standard Deviation": std,
        "Skewness": skew,
        "Kurtosis": kurt,
        "Line Length": line_len,
        "Hjorth Activity": activity,
        "Hjorth Mobility": mobility,
        "Hjorth Complexity": complexity,
    }


def compute_features_for_window(seg, sf):
    """
    For each channel in the window, compute time + frequency features.
    Returns dict: ch1_Delta Bandpower, ch1_Mean, ..., ch2_Delta Bandpower, ...
    """
    n_ch, _ = seg.shape
    feat_dict = {}

    for ch in range(n_ch):
        x = seg[ch, :]

        bp_dict = compute_bandpowers_and_relative(x, sf)
        td_dict = compute_time_features(x)

        prefix = f"ch{ch+1}_"

        for k, v in {**bp_dict, **td_dict}.items():
            feat_dict[prefix + k] = v

    return feat_dict


# =========================================
# 5) process_one_run: preprocessing + per-window features
# =========================================
def process_one_run(subject, session, run, runs_df):

    print("\n====================================")
    print(f"Processing {subject} | {session} | run-{run:02d}")
    print("====================================")

    run_base = f"{subject}_{session}_task-szMonitoring_run-{run:02d}"

    eeg_path = BASE_DIR / subject / session / "eeg" / f"{run_base}_eeg.edf"
    ecg_path = BASE_DIR / subject / session / "ecg" / f"{run_base}_ecg.edf"
    emg_path = BASE_DIR / subject / session / "emg" / f"{run_base}_emg.edf"

    if not eeg_path.exists():
        return None

    try:
        eeg_raw = mne.io.read_raw_edf(eeg_path, preload=True, verbose=False)
    except:
        return None

    ecg_raw = mne.io.read_raw_edf(ecg_path, preload=True, verbose=False) if ecg_path.exists() else None
    emg_raw = mne.io.read_raw_edf(emg_path, preload=True, verbose=False) if emg_path.exists() else None

    # Filtering
    eeg_bp = eeg_raw.copy()
    eeg_bp.notch_filter(freqs=[50.0], verbose=False)
    eeg_bp.filter(l_freq=0.5, h_freq=45.0, verbose=False)

    # Build state mask
    state_mask = build_state_mask_from_runs(
        eeg_bp, subject=subject, session=session, run=run, runs_df=runs_df, preictal_sec=PREICTAL_SEC
    )

    # ECG regression
    if ecg_raw is not None and has_ecg_artefact(eeg_bp, ecg_raw):
        eeg_ecg = regression_artifact_removal(eeg_bp, ecg_raw, ref_name="ECG")
    else:
        eeg_ecg = eeg_bp.copy()

    # EMG wavelet denoising
    if has_emg_artefact(eeg_ecg):
        eeg_clean = wavelet_denoise_emg_raw(eeg_ecg)
    else:
        eeg_clean = eeg_ecg.copy()

    # Windowing
    X, y = window_and_label(eeg_clean, state_mask, win_sec=WINDOW_SEC)
    if X is None or y is None:
        return None

    # Print counts
    n_normal   = int(np.sum(y == 0))
    n_preictal = int(np.sum(y == 1))
    n_ictal    = int(np.sum(y == 2))

    print(f"🟦  Normal: {n_normal} windows")
    print(f"🟧  Pre-ictal: {n_preictal} windows")
    print(f"🟥  Ictal: {n_ictal} windows")
    print(f"\n➡ Finished run with Total windows: {len(y)}\n")

    # Feature extraction
    sf = float(eeg_clean.info["sfreq"])
    rows = []
    for w in range(X.shape[0]):
        seg = X[w, :, :]
        feat_dict = compute_features_for_window(seg, sf)
        row = {"subject": subject, "run": int(run), "window_idx": int(w), "label": int(y[w])}
        row.update(feat_dict)
        rows.append(row)

    return pd.DataFrame(rows)



# =========================================
# 6) Prepare unique runs and select Batch 1
# =========================================

# sub_id = numeric part of subject name (sub-001 → 1)
runs_table["sub_id"] = runs_table["subject"].str.extract(r"(\d+)").astype(int)

# Unique runs only
runs_unique = runs_table[["subject", "session", "run", "sub_id"]].drop_duplicates()

print("🔢 Total unique runs in CSV:", len(runs_unique))

# Batch 1: runs from index 200 to 394 (i.e., 201–395)
batch1 = runs_unique.drop(columns=["sub_id"]).iloc[200:395].copy()

print("✅ Batch 1 unique runs:", len(batch1))
display(batch1.head())


# =========================================
# 7) Loop over Batch 1 → extract window features and save to CSV
# =========================================
first = True

for idx, row in batch1.iterrows():
    subject = str(row["subject"]).strip()
    session = str(row["session"]).strip()
    run     = int(row["run"])

    print(f"\n➡ Running {subject} | {session} | run-{run:02d}")
    df_run = process_one_run(subject, session, run, runs_df=runs_table)

    if df_run is not None and not df_run.empty:
        if first:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="w")
            first = False
        else:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="a", header=False)

print("\n🎉 DONE! Saved Batch 1 per-window EEG features to:")
print(OUT_FEAT_CSV)


BASE_DIR: C:\SeniorProject_Data
RUNS_CSV exists: True
Runs table shape: (883, 6)


,subject,session,run,onset,duration,eventType
0,sub-001,ses-01,3,57975.0,72.0,sz_foc_ia_nm
1,sub-001,ses-01,5,60968.0,81.0,sz_foc_a_nm
2,sub-001,ses-01,7,16300.0,143.0,sz_foc_ia_m_tonic
3,sub-001,ses-01,8,28285.0,25.0,sz_foc_a_m_automatisms
4,sub-002,ses-01,1,4046.0,20.0,sz_foc_a_nm


🔢 Total unique runs in CSV: 395
✅ Batch 1 unique runs (first 200): 195


,subject,session,run
552,sub-073,ses-01,50
555,sub-073,ses-01,51
564,sub-073,ses-01,52
565,sub-073,ses-01,54
566,sub-074,ses-01,2



➡ Running sub-073 | ses-01 | run-50

Processing sub-073 | ses-01 | run-50
[ECG] max |corr(EEG, ECG)| = 0.004
[EMG] median HF(40–100Hz)/total power = 0.036
[window_and_label] X shape = (10779, 2, 512)
[window_and_label] label counts = {np.int8(0): np.int64(9261), np.int8(1): np.int64(1350), np.int8(2): np.int64(168)}
🟦  Normal: 9261 windows
🟧  Pre-ictal: 1350 windows
🟥  Ictal: 168 windows

➡ Finished run with Total windows: 10779


➡ Running sub-073 | ses-01 | run-51

Processing sub-073 | ses-01 | run-51
[ECG] max |corr(EEG, ECG)| = 0.017
[EMG] median HF(40–100Hz)/total power = 0.041
[window_and_label] X shape = (10773, 2, 512)
[window_and_label] label counts = {np.int8(0): np.int64(7247), np.int8(1): np.int64(3341), np.int8(2): np.int64(185)}
🟦  Normal: 7247 windows
🟧  Pre-ictal: 3341 windows
🟥  Ictal: 185 windows

➡ Finished run with Total windows: 10773


➡ Running sub-073 | ses-01 | run-52

Processing sub-073 | ses-01 | run-52
[ECG] max |corr(EEG, ECG)| = 0.001
[EMG] median HF(40–1

##### RUN: 1-200

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import mne
import pywt

mne.set_log_level("WARNING")

# =========================================
# 0) Paths + constants
# =========================================

# Be careful with the data path (it contains SeizeIT2\SeizeIT2)
BASE_DIR = Path(r"C:\SeniorProject_Data")

# Runs + seizures file (CSV with onset/duration for each run)
RUNS_CSV = Path(r"C:\SeniorProject\Good_Data\seizure_runs_details.csv")

# Output file name for Batch 1
OUT_FEAT_CSV = Path(r"C:\SeniorProject\Good_Data\MOV_Good_Runs_Batch1-200.csv")

# Constants
WINDOW_SEC   = 2.0          # Window length in seconds
PREICTAL_SEC = 15 * 60      # 15 minutes before the seizure (for label=1)

print("BASE_DIR:", BASE_DIR)
print("RUNS_CSV exists:", RUNS_CSV.exists())

# Read runs + seizures table from CSV
runs_table = pd.read_csv(RUNS_CSV)
runs_table["subject"] = runs_table["subject"].astype(str).str.strip()   # e.g. sub-001
runs_table["session"] = runs_table["session"].astype(str).str.strip()   # e.g. ses-01
runs_table["run"]     = pd.to_numeric(runs_table["run"], errors="raise").astype(int)

print("Runs table shape:", runs_table.shape)
display(runs_table.head())


# =========================================
# 1) Build state mask from runs file (instead of events.tsv)
# =========================================
def build_state_mask_from_runs(raw, subject, session, run, runs_df, preictal_sec=PREICTAL_SEC):
    sf = raw.info["sfreq"]
    n_samples = raw.n_times
    mask = np.zeros(n_samples, dtype=np.int8)

    sub_df = runs_df[
        (runs_df["subject"] == subject) &
        (runs_df["session"] == session) &
        (runs_df["run"] == int(run))
    ].copy()

    if sub_df.empty:
        return mask

    sub_df["onset"]    = pd.to_numeric(sub_df["onset"], errors="coerce")
    sub_df["duration"] = pd.to_numeric(sub_df["duration"], errors="coerce")
    sub_df = sub_df.dropna(subset=["onset", "duration"])

    if sub_df.empty:
        return mask

    for _, row in sub_df.iterrows():
        onset_sec = float(row["onset"])
        dur_sec   = float(row["duration"])

        ictal_start = int(onset_sec * sf)
        ictal_end   = int((onset_sec + dur_sec) * sf)

        ictal_start = max(0, min(ictal_start, n_samples))
        ictal_end   = max(0, min(ictal_end,   n_samples))

        pre_start = max(0, ictal_start - int(preictal_sec * sf))

        mask[pre_start:ictal_start] = 1
        mask[ictal_start:ictal_end] = 2

    return mask



# =========================================
# 2) ECG/EMG artefact helpers
# =========================================
def prepare_ref_for_regression(eeg_raw, ref_raw, ref_name="REF"):
    if ref_raw is None:
        print(f"[{ref_name}] No reference file.")
        return None

    sf_eeg = eeg_raw.info["sfreq"]
    sf_ref = ref_raw.info["sfreq"]

    ref = ref_raw.copy()
    if sf_ref != sf_eeg:
        print(f"[{ref_name}] Resampling reference from {sf_ref} Hz to {sf_eeg} Hz (ref only).")
        ref.resample(sfreq=sf_eeg, npad="auto")

    eeg_len = eeg_raw.n_times
    ref_data = ref.get_data()
    ref_len = ref_data.shape[1]

    if ref_len > eeg_len:
        ref_data = ref_data[:, :eeg_len]
    elif ref_len < eeg_len:
        pad = np.zeros((ref_data.shape[0], eeg_len - ref_len), dtype=ref_data.dtype)
        ref_data = np.concatenate([ref_data, pad], axis=1)

    if np.var(ref_data) < 1e-10:
        print(f"[{ref_name}] Reference almost flat → skip.")
        return None

    return ref_data


def regression_artifact_removal(eeg_raw, ref_raw, ref_name="REF"):
    ref_data = prepare_ref_for_regression(eeg_raw, ref_raw, ref_name=ref_name)
    if ref_data is None:
        print(f"[{ref_name}] Regression skipped.")
        return eeg_raw.copy()

    Y = eeg_raw.get_data()
    X_ref = ref_data

    X = np.vstack([X_ref, np.ones((1, X_ref.shape[1]))]).T
    Y_T = Y.T

    beta, _, _, _ = np.linalg.lstsq(X, Y_T, rcond=None)
    artefact = (X @ beta).T
    cleaned  = Y - artefact

    out = eeg_raw.copy()
    out._data = cleaned
    print(f"[{ref_name}] Regression artefact removal done.")
    return out


def has_ecg_artefact(eeg_raw, ecg_raw, corr_thresh=0.3):
    ref_data = prepare_ref_for_regression(eeg_raw, ecg_raw, ref_name="ECG")
    if ref_data is None:
        return False

    eeg_data = eeg_raw.get_data()
    ref_ch   = ref_data[0]

    corrs = []
    for ch_idx in range(eeg_data.shape[0]):
        sig = eeg_data[ch_idx]
        c = np.corrcoef(sig, ref_ch)[0, 1]
        if not np.isnan(c):
            corrs.append(abs(c))

    if not corrs:
        return False

    max_corr = max(corrs)
    print(f"[ECG] max |corr(EEG, ECG)| = {max_corr:.3f}")
    return max_corr >= corr_thresh


def has_emg_artefact(eeg_raw, high_freq_band=(40.0, 100.0), ratio_thresh=0.3):
    data = eeg_raw.get_data()
    sf   = eeg_raw.info["sfreq"]

    hf = mne.filter.filter_data(
        data, sf,
        l_freq=high_freq_band[0],
        h_freq=high_freq_band[1],
        verbose=False
    )

    hf_power    = np.mean(hf**2, axis=1)
    total_power = np.mean(data**2, axis=1) + 1e-12
    ratios = hf_power / total_power
    median_ratio = float(np.median(ratios))

    print(f"[EMG] median HF(40–100Hz)/total power = {median_ratio:.3f}")
    return median_ratio >= ratio_thresh


def wavelet_denoise_emg_raw(eeg_raw, wavelet="db4", level=2, thr_scale=0.5):
    data = eeg_raw.get_data()
    denoised = np.zeros_like(data)

    for ch in range(data.shape[0]):
        sig = data[ch, :]
        coeffs = pywt.wavedec(sig, wavelet, level=level)
        sigma = np.median(np.abs(coeffs[-1])) / 0.6745
        thr = thr_scale * sigma * np.sqrt(2 * np.log(len(sig)))

        new_coeffs = [coeffs[0]]
        for d in coeffs[1:]:
            new_d = pywt.threshold(d, thr, mode="soft")
            new_coeffs.append(new_d)

        recon = pywt.waverec(new_coeffs, wavelet)
        if len(recon) > len(sig):
            recon = recon[:len(sig)]
        elif len(recon) < len(sig):
            pad = np.zeros(len(sig) - len(recon))
            recon = np.concatenate([recon, pad])
        denoised[ch, :] = recon

    out = eeg_raw.copy()
    out._data = denoised
    print("[EMG] Wavelet denoising done.")
    return out


# =========================================
# 3) window_and_label → PER-WINDOW (priority: 2 then 1 then 0)
# =========================================
def window_and_label(eeg_raw, state_mask, win_sec=WINDOW_SEC):
    """
    - Split the signal into 2-second windows (no overlap)
    - Assign a window label based on the state_mask inside it
    """
    sf = eeg_raw.info["sfreq"]
    data = eeg_raw.get_data()
    n_ch, n_samples = data.shape

    win_samples = int(win_sec * sf)
    if n_samples < win_samples:
        print("⚠ Run too short.")
        return None, None

    mask = np.asarray(state_mask, dtype=np.int8)
    if mask.size < n_samples:
        pad = np.zeros(n_samples - mask.size, dtype=np.int8)
        mask = np.concatenate([mask, pad])
    elif mask.size > n_samples:
        mask = mask[:n_samples]

    starts = np.arange(0, n_samples - win_samples + 1, win_samples)

    X_list, y_list = [], []
    for s in starts:
        e = s + win_samples
        seg = data[:, s:e]
        seg_mask = mask[s:e]

        if seg_mask.size == 0:
            lab = 0
        else:
            if (seg_mask == 2).any():
                lab = 2
            elif (seg_mask == 1).any():
                lab = 1
            else:
                lab = 0

        X_list.append(seg)
        y_list.append(lab)

    if not X_list:
        return None, None

    X = np.stack(X_list, axis=0)          # (n_windows, n_channels, n_samples)
    y = np.array(y_list, dtype=np.int8)   # (n_windows,)

    print(f"[window_and_label] X shape = {X.shape}")
    print(f"[window_and_label] label counts = {dict(zip(*np.unique(y, return_counts=True)))}")
    return X, y


# =========================================
# 4) Feature helpers PER-CHANNEL
# =========================================
def compute_bandpowers_and_relative(x, sf):
    """
    - Absolute bandpowers:
        Delta (0.5–4), Theta (4–8), Alpha (8–12),
        Beta (12–30), Gamma (30–45)
    - Relative bandpowers = abs / sum(abs)
    """
    x = np.asarray(x, dtype=float)
    n = x.size

    freqs = np.fft.rfftfreq(n, d=1.0 / sf)
    fft_vals = np.fft.rfft(x)
    psd = (np.abs(fft_vals) ** 2) / n  # power spectrum

    def band_power(fmin, fmax):
        mask = (freqs >= fmin) & (freqs < fmax)
        if not np.any(mask):
            return 0.0
        return psd[mask].sum()

    bp_delta = band_power(0.5, 4.0)
    bp_theta = band_power(4.0, 8.0)
    bp_alpha = band_power(8.0, 12.0)
    bp_beta  = band_power(12.0, 30.0)
    bp_gamma = band_power(30.0, 45.0)

    total_bp = bp_delta + bp_theta + bp_alpha + bp_beta + bp_gamma + 1e-12

    return {
        "Delta Bandpower": bp_delta,
        "Theta Bandpower": bp_theta,
        "Alpha Bandpower": bp_alpha,
        "Beta Bandpower":  bp_beta,
        "Gamma Bandpower": bp_gamma,

        "Relative Delta Bandpower": bp_delta / total_bp,
        "Relative Theta Bandpower": bp_theta / total_bp,
        "Relative Alpha Bandpower": bp_alpha / total_bp,
        "Relative Beta Bandpower":  bp_beta  / total_bp,
        "Relative Gamma Bandpower": bp_gamma / total_bp,
    }


def compute_time_features(x):
    """
    - Interquartile Range
    - Median Absolute Deviation
    - Mean
    - Median
    - Variance
    - Entropy
    - Standard Deviation
    - Skewness
    - Kurtosis
    - Line Length
    - Hjorth Parameters: activity, mobility, complexity
    """
    x = np.asarray(x, dtype=float)

    mean = np.mean(x)
    median = np.median(x)
    var = np.var(x)
    std = np.sqrt(var + 1e-12)

    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1

    mad = np.median(np.abs(x - median))

    # Entropy
    if np.allclose(x, x[0]):
        entropy = 0.0
    else:
        hist, _ = np.histogram(x, bins=50, density=True)
        p = hist + 1e-12
        p = p / p.sum()
        entropy = -np.sum(p * np.log2(p))

    centered = x - mean
    m3 = np.mean(centered**3)
    m4 = np.mean(centered**4)
    skew = m3 / (std**3 + 1e-12)
    kurt = m4 / (std**4 + 1e-12)

    line_len = np.sum(np.abs(np.diff(x)))

    activity = var
    dx = np.diff(x)
    var_dx = np.var(dx)
    mobility = np.sqrt(var_dx / (var + 1e-12))

    ddx = np.diff(dx)
    var_ddx = np.var(ddx) if ddx.size > 0 else 0.0
    mobility_dx = np.sqrt(var_ddx / (var_dx + 1e-12)) if var_dx > 0 else 0.0
    complexity = mobility_dx / (mobility + 1e-12)

    return {
        "Interquartile Range": iqr,
        "Median Absolute Deviation": mad,
        "Mean": mean,
        "Median": median,
        "Variance": var,
        "Entropy": entropy,
        "Standard Deviation": std,
        "Skewness": skew,
        "Kurtosis": kurt,
        "Line Length": line_len,
        "Hjorth Activity": activity,
        "Hjorth Mobility": mobility,
        "Hjorth Complexity": complexity,
    }


def compute_features_for_window(seg, sf):
    """
    seg: (n_channels, n_samples) window
    Returns a dict with features for each channel, e.g.:
      ch1_Delta Bandpower, ch2_Delta Bandpower, ...
    """
    n_ch, _ = seg.shape
    feat_dict = {}

    for ch in range(n_ch):
        x = seg[ch, :]

        bp_dict = compute_bandpowers_and_relative(x, sf)
        td_dict = compute_time_features(x)

        prefix = f"ch{ch+1}_"

        for k, v in {**bp_dict, **td_dict}.items():
            feat_dict[prefix + k] = v

    return feat_dict


# =========================================
# 5) process_one_run function: preprocessing + per-window features
# =========================================
def process_one_run(subject, session, run, runs_df):

    print("\n====================================")
    print(f"Processing {subject} | {session} | run-{run:02d}")
    print("====================================")

    run_base = f"{subject}_{session}_task-szMonitoring_run-{run:02d}"

    eeg_path = BASE_DIR / subject / session / "eeg" / f"{run_base}_eeg.edf"
    ecg_path = BASE_DIR / subject / session / "ecg" / f"{run_base}_ecg.edf"
    emg_path = BASE_DIR / subject / session / "emg" / f"{run_base}_emg.edf"

    if not eeg_path.exists():
        return None

    try:
        eeg_raw = mne.io.read_raw_edf(eeg_path, preload=True, verbose=False)
    except:
        return None

    ecg_raw = mne.io.read_raw_edf(ecg_path, preload=True, verbose=False) if ecg_path.exists() else None
    emg_raw = mne.io.read_raw_edf(emg_path, preload=True, verbose=False) if emg_path.exists() else None

    # Notch + Bandpass without printing
    eeg_bp = eeg_raw.copy()
    eeg_bp.notch_filter(freqs=[50.0], verbose=False)
    eeg_bp.filter(l_freq=0.5, h_freq=45.0, verbose=False)

    # State mask ONLY (no prints)
    state_mask = build_state_mask_from_runs(
        eeg_bp, subject=subject, session=session, run=run, runs_df=runs_df, preictal_sec=PREICTAL_SEC
    )

    # ECG regression without prints
    if ecg_raw is not None and has_ecg_artefact(eeg_bp, ecg_raw):
        eeg_ecg = regression_artifact_removal(eeg_bp, ecg_raw, ref_name="ECG")
    else:
        eeg_ecg = eeg_bp.copy()

    # EMG wavelet denoise without prints
    if has_emg_artefact(eeg_ecg):
        eeg_clean = wavelet_denoise_emg_raw(eeg_ecg)
    else:
        eeg_clean = eeg_ecg.copy()

    # Windowing
    X, y = window_and_label(eeg_clean, state_mask, win_sec=WINDOW_SEC)
    if X is None or y is None:
        return None

    # ======= ONLY the output you want =======
    n_normal   = int(np.sum(y == 0))
    n_preictal = int(np.sum(y == 1))
    n_ictal    = int(np.sum(y == 2))

    print(f"🟦  Normal: {n_normal} windows")
    print(f"🟧  Pre-ictal: {n_preictal} windows")
    print(f"🟥  Ictal: {n_ictal} windows")
    print(f"\n➡ Finished run with Total windows: {len(y)}\n")
    # ========================================

    # Feature extraction (no prints)
    sf = float(eeg_clean.info["sfreq"])
    rows = []
    for w in range(X.shape[0]):
        seg = X[w, :, :]
        feat_dict = compute_features_for_window(seg, sf)
        row = {"subject": subject, "run": int(run), "window_idx": int(w), "label": int(y[w])}
        row.update(feat_dict)
        rows.append(row)

    return pd.DataFrame(rows)




# =========================================
# 6) Prepare unique runs and split into Batch 1
# =========================================

# sub_id = numeric part inside subject name (sub-001 → 1)
runs_table["sub_id"] = runs_table["subject"].str.extract(r"(\d+)").astype(int)

# Unique runs only
runs_unique = runs_table[["subject", "session", "run", "sub_id"]].drop_duplicates()

print("🔢 Total unique runs in CSV:", len(runs_unique))

# ✅ Batch 1: first 200 unique runs only
batch1 = runs_unique.drop(columns=["sub_id"]).iloc[1:200].copy()

print("✅ Batch 1 unique runs (first 200):", len(batch1))
display(batch1.head())

# =========================================
# 7) Loop over batch1 only → extract FEATURES and write to CSV
# =========================================
first = True

for idx, row in batch1.iterrows():
    subject = str(row["subject"]).strip()
    session = str(row["session"]).strip()
    run     = int(row["run"])

    print(f"\n➡ Running {subject} | {session} | run-{run:02d}")
    df_run = process_one_run(subject, session, run, runs_df=runs_table)

    if df_run is not None and not df_run.empty:
        if first:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="w")
            first = False
        else:
            df_run.to_csv(OUT_FEAT_CSV, index=False, encoding="utf-8-sig", mode="a", header=False)

print("\n🎉 DONE! Saved Batch 1 per-window EEG features to:")
print(OUT_FEAT_CSV)


BASE_DIR: C:\SeniorProject_Data
RUNS_CSV exists: True
Runs table shape: (883, 6)


,subject,session,run,onset,duration,eventType
0,sub-001,ses-01,3,57975.0,72.0,sz_foc_ia_nm
1,sub-001,ses-01,5,60968.0,81.0,sz_foc_a_nm
2,sub-001,ses-01,7,16300.0,143.0,sz_foc_ia_m_tonic
3,sub-001,ses-01,8,28285.0,25.0,sz_foc_a_m_automatisms
4,sub-002,ses-01,1,4046.0,20.0,sz_foc_a_nm


🔢 Total unique runs in CSV: 395
✅ Batch 1 unique runs (first 200): 199


,subject,session,run
1,sub-001,ses-01,5
2,sub-001,ses-01,7
3,sub-001,ses-01,8
4,sub-002,ses-01,1
6,sub-002,ses-01,2



➡ Running sub-001 | ses-01 | run-05

Processing sub-001 | ses-01 | run-05
[ECG] max |corr(EEG, ECG)| = 0.006
[EMG] median HF(40–100Hz)/total power = 0.030
[window_and_label] X shape = (31255, 2, 512)
[window_and_label] label counts = {np.int8(0): np.int64(30764), np.int8(1): np.int64(450), np.int8(2): np.int64(41)}
🟦  Normal: 30764 windows
🟧  Pre-ictal: 450 windows
🟥  Ictal: 41 windows

➡ Finished run with Total windows: 31255


➡ Running sub-001 | ses-01 | run-07

Processing sub-001 | ses-01 | run-07
[ECG] max |corr(EEG, ECG)| = 0.008
[EMG] median HF(40–100Hz)/total power = 0.053
[window_and_label] X shape = (14759, 2, 512)
[window_and_label] label counts = {np.int8(0): np.int64(14237), np.int8(1): np.int64(450), np.int8(2): np.int64(72)}
🟦  Normal: 14237 windows
🟧  Pre-ictal: 450 windows
🟥  Ictal: 72 windows

➡ Finished run with Total windows: 14759


➡ Running sub-001 | ses-01 | run-08

Processing sub-001 | ses-01 | run-08
[ECG] max |corr(EEG, ECG)| = 0.003
[EMG] median HF(40–100Hz